In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'>

# LCMS Map Assemblage

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>


## Overview


This notebook teaches how to take raw model outputs and assemble final map classes


### Objective

In this tutorial, you learn how to manipulate raw model output GEE image arrays to create map outputs of a single class.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Looking at raw GEE image array model output assets
- Manipulating the image arrays for a basic map assemblage
- Performing a more complicated map assemblage that balances omission and commission for Change

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator
import matplotlib.pyplot as plt
import pandas as pd  
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
PyTables is not installed. No support for HDF output.
Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [3]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [4]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_rawLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-5_rawLCMSOutputs'

export_assembledLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-6_assembledLCMSOutputs'


aml.create_asset(export_assembledLCMSOutputs_collection, asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_assembledLCMSOutputs_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-6_assembledLCMSOutputs']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs
Done


In [6]:
Map.proxy_url = workbench_url
Map.clearMap()
# Bring in raw LCMS model outputs
raw_lcms = ee.ImageCollection(export_rawLCMSOutputs_collection)

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2020-6").first().toDictionary().getInfo()
# print(lcms_viz_dict)
    
# Get some info
products = raw_lcms.aggregate_histogram('product').keys().getInfo()


# Specify missing classes in PRUSVI
# Talls Shrubs (2 and 6)  and snow/ice (13) are not present and need filled back in
# Change and Land_Use have all values
missing_names = {'Change':[],
                  'Land_Cover':['Tall Shrubs & Trees Mix (SEAK Only)',
                                'Tall Shrubs (SEAK Only)',
                                'Snow or Ice'],
                  'Land_Use':[]
                 }
def arrayToImage(img,bandNames):
    return img.arrayProject([0])\
            .arrayFlatten([bandNames])\
            .copyProperties(img,['system:time_start'])

# var c = ee.ImageCollection('projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs')
        # .filter(ee.Filter.eq('product','Land_Cover'))

# Function to add in a fill value in a 1-d image array of a given index and value
def fillArray(img,index,fillValue=0):
    imgLeft = img.arraySlice(0,0,index)
    imgRight = img.arraySlice(0,index,None)
    imgLeft = imgLeft.arrayCat(ee.Image(fillValue).toArray(),0)
    out = imgLeft.arrayCat(imgRight,0)
    return ee.Image(out.copyProperties(img).copyProperties(img,['system:time_start']))

# Function to handle filling multiple values
def fillArrayMulti(img,indices,fillValue):
    for index in indices:
        img = fillArray(img,index,fillValue)
    return img

# Get the class code of the most confident class
# This is the most basic assemblage method and most commonly used
# We will start with this approach and then illustrate when this simple approach may not 
# create the best map in some instances
def getMostProbableClass(raw_lcms_product_yr,product):
    # Pull the index of the most probable value
    # Since 0 is not used for LCMS outputs, add 1
    max_prob_class = raw_lcms_product_yr.arrayArgmax().arrayGet(0).add(1).byte().rename([product])
    max_prob_class = ee.Image(max_prob_class)

    null_code = lcms_viz_dict[f'{product}_class_values'][-1]
    max_prob_class = max_prob_class.unmask(null_code)
    return max_prob_class.copyProperties(raw_lcms_product_yr,['system:time_start'])

# Iterate across each product and assemble the most probable class collection from the raw
for product in products:
    # Pull the class names
    class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
   
    product_title = product.replace('_',' ')
    
    # Filter to only include the product of interest
    raw_lcms_product = raw_lcms.filter(ee.Filter.eq('product',product))
    
    # Fill in any missing values
    missing_names_product = missing_names[product]
    missing_indices_product = [lcms_viz_dict[f'{product}_class_names'].index(missing_name_product) for missing_name_product in missing_names_product]
    
    if len(missing_names_product) > 0:
        raw_lcms_product = raw_lcms_product.map(lambda img: fillArrayMulti(img,missing_indices_product,0))
    
    # Find the max probability class
    maxProb_lcms_product = raw_lcms_product.map(lambda raw_lcms_product_yr:getMostProbableClass(raw_lcms_product_yr,product).set(lcms_viz_dict))
    
    # Convert raw model probabilities to bands for exploration
    raw_lcms_product = raw_lcms_product.map(lambda img:arrayToImage(img,class_names))
    
    # Add the raw model probabilities to query
    Map.addTimeLapse(raw_lcms_product,{},f'Raw LCMS {product_title}')
    
    # Add the assembed final class maps
    Map.addTimeLapse(maxProb_lcms_product,{'autoViz':True},f'Most Probable LCMS {product_title}')
Map.turnOnInspector()
Map.view()
# Take note that the Raw LCMS Change probabilities generally have stable as the most probable
# We will need a different assemblage method for change
# print('Done')

Adding layer: Raw LCMS Change
Adding layer: Most Probable LCMS Change
Adding layer: Raw LCMS Land Cover
Adding layer: Most Probable LCMS Land Cover
Adding layer: Raw LCMS Land Use
Adding layer: Most Probable LCMS Land Use
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:1235/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  1235
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [22/Aug/2023 03:18:54] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [37]:
Map.clearMap()
# While this assemblage method works well, you can introduce various rules to talor the final 
# map product to different audiences
# In the case of LCMS, there is a focus on forest applications
# We know that our models are generally not very confident about classifying change largely
# due to the limited number of training samples that experience change
# We also know our land use classes are not mutually exclusive (e.g. non-forest wetlands can potentially occur in any land use)
# Therefore we have a lot of confusion in our land use outputs

# For assembling change, we choose thresholds that balance omission and commission
# These were computed in module 5.1
change_thresholds = { 'Slow Loss': 0.29, 'Fast Loss': 0.28, 'Gain': 0.36}

# Since a pixel may have probabilities above the threshold of more than one class, we choose the class
# with the model confidence that is the highest probability that is also above its respective threshold. 
# e.g. if the fast loss probability is 0.5 and the slow loss probability is 0.35, the pixel will be assigned fast loss
# We will not include stable since stable will be any pixel that is not already assigned to one of the 3 change classes

product = 'Change'

# Pull the class names
class_names = lcms_viz_dict[f'{product}_class_names']
class_codes = lcms_viz_dict[f'{product}_class_values']
class_dict = dict(zip(class_names,class_codes))

# Pull the codes in the correct order
class_order_codes = [class_dict[cl] for cl in list(change_thresholds.keys())]

# Filter to only include the product of interest
raw_lcms_product = raw_lcms.filter(ee.Filter.eq('product',product))
    
# Function to assemble change for a given year
# Uses a specified set of bands and thresholds in order to find the highest probability class above its threshold
# It then fills in the stable value as any remaining non null value
# Nulls are then recoded to the non processing code
def assembleChange(img,product):
    # Convert array to multi-band image
    img = ee.Image(arrayToImage(img,lcms_viz_dict[f'{product}_class_names'][:-1]))
    
    # Select only the bands we'd like to assemble
    img = img.select(list(change_thresholds.keys()))
    
    # Find the most confident class
    maxConf = img.reduce(ee.Reducer.max())
    
    # Find pixels that are the most confident and above the threshold
    maxConfMask = img.eq(maxConf).And(img.gte(list(change_thresholds.values()))).selfMask()
    
    # Get class code for any pixel that is above its threshold for the most confident class
    maxClass = ee.Image(class_order_codes).updateMask(maxConfMask).reduce(ee.Reducer.first())
    
    # Fill stable back in
    maxClass = maxClass.unmask(1)
    
    # Burn in non processing where the input was null
    maxClass = maxClass.where(img.mask().reduce(ee.Reducer.min()).Not(),class_codes[-1])
    
    return maxClass.rename([product]).copyProperties(img).copyProperties(img,['system:time_start'])

# Iterate across the change probabilities and assemble final change class
assembledChange = raw_lcms_product.map(lambda img:assembleChange(img,product).set(lcms_viz_dict))

# Explore it on the map
Map.addTimeLapse(assembledChange,{'autoViz':True},'Assembled LCMS Change')

Map.turnOnInspector()
Map.view()

Adding layer: Assembled LCMS Change
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 02:06:53] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [36]:
Map.clearMap()
# Viewing change outputs can be easier as a composite that shows the most recent year of a single change type
# This example will show the most recent year of fast loss

# First, add a year constant band
assembledChange = assembledChange.map(getImagesLib.addYearBand)

# Then mask where change equals fast loss (3)
fastLoss = assembledChange.map(lambda img: img.updateMask(img.select([0]).eq(3)))

# Find the most recent year
mostRecentFastLoss = fastLoss.max().select(['year'])
Map.addLayer(mostRecentFastLoss,{'min':1985,'max':2022,'palette':changeDetectionLib.lossYearPalette},'Most Recent Year of Fast Loss')

Map.turnOnInspector()
Map.view()
print(assembledChange.first().bandNames().getInfo())

Adding layer: Most Recent Year of Fast Loss
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


['Change', 'year', 'year_1']


127.0.0.1 - - [23/Aug/2023 02:01:17] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [40]:
Map.clearMap()
# Now lets export assembled assets

# OPTIONAL!!!!
# Optionally, we can export using the tile grid approach
# For PRUSVI, LCMS does not need to export using this approach, but this is how you would set it up
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary')

# Set the size (in meters) of the tiles
# We can likely use a large tile for this step
# If exports fail, reducing the tileSize is likely to help
tileSize = 240000


# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)


# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(grid)
Map.view()

Adding layer: Tile Grid 240000m
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 02:08:35] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


* Iterate across each year and assemble full stack output
* [Example of fully assembed bands](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2022-8#bands)
* We will not be producing the qa-bits bands

In [ ]:
# Iterate across each year, assemble full stack output 
# products = ['Land_Cover','Land_Use']
Map.clearMap()

# Provide a study area name and version
study_area_name = 'PRUSVI'
version = '2022-Training'

# Set up years to apply models across
apply_years = list(range(1985,2022+1))


# This will be populated on the first iteration
pyramidingPolicy = None


for apply_year in apply_years:
    assembled_list = []
    probability_list = []
    for product in products:
        # Pull the class names
        class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
        
        # Get rid of missing class names
        class_names = [nm for nm in class_names if nm not in missing_names[product]]
        
        # Get rid of any characters that aren't allowed
        class_names = [nm.replace(' ','-').replace('/','-').replace('&','and') for nm in class_names]
        
        probability_names = [f'{product}_Raw_Probability_{class_name}' for class_name in class_names]
        
        product_title = product.replace('_',' ')

        # Filter to only include the product of interest
        raw_lcms_product_yr = raw_lcms\
                            .filter(ee.Filter.calendarRange(apply_year,apply_year,'year'))\
                            .filter(ee.Filter.eq('product',product)).first()
        
        if product in ['Land_Cover','Land_Use']:
            # Find the max probability class
            maxProb_lcms_product_yr = getMostProbableClass(raw_lcms_product_yr,product)
            assembled_list.append(maxProb_lcms_product_yr)
        
        elif product == 'Change':
            change_lcms_product_yr = assembleChange(raw_lcms_product_yr,product)
            assembled_list.append(change_lcms_product_yr)
            
        # Convert raw model probabilities to bands for exploration
        prob_lcms_product_yr = arrayToImage(raw_lcms_product_yr,probability_names)
        probability_list.append(prob_lcms_product_yr)
        
    final_stack = ee.Image(ee.Image.cat([ee.Image.cat(assembled_list).byte(),ee.Image.cat(probability_list).multiply(100)]).byte()\
                    .copyProperties(raw_lcms_product_yr,['study_area','year']))

    exportName = f'LCMS_{study_area_name}_v{version}_{apply_year}'
    exportPath = f'{export_assembledLCMSOutputs_collection}/{exportName}'

    # Set up a pyramiding object for each band
    # Final output bands are thematic and need a mode resampling method
    # Probability bands are continuous, so mean is appropriate
    if pyramidingPolicy == None:
        bns = final_stack.bandNames().getInfo()
        pyramidingPolicy = {}
        for bn in bns:
            if bn.find('_Probability_') == -1:
                pyramidingPolicy[bn] = 'mode'
            else:
                pyramidingPolicy[bn] = 'mean'
   
    getImagesLib.exportToAssetWrapper(final_stack,exportName,exportPath,pyramidingPolicy,studyArea,scale,crs,transform,overwrite=True)

print('Done')

Exporting: LCMS_PRUSVI_v2022-Training_1985
Exporting: LCMS_PRUSVI_v2022-Training_1986
Exporting: LCMS_PRUSVI_v2022-Training_1987
Exporting: LCMS_PRUSVI_v2022-Training_1988
Exporting: LCMS_PRUSVI_v2022-Training_1989
Exporting: LCMS_PRUSVI_v2022-Training_1990
Exporting: LCMS_PRUSVI_v2022-Training_1991
Exporting: LCMS_PRUSVI_v2022-Training_1992
Exporting: LCMS_PRUSVI_v2022-Training_1993
Exporting: LCMS_PRUSVI_v2022-Training_1994
Exporting: LCMS_PRUSVI_v2022-Training_1995
Exporting: LCMS_PRUSVI_v2022-Training_1996
Exporting: LCMS_PRUSVI_v2022-Training_1997
Exporting: LCMS_PRUSVI_v2022-Training_1998
Exporting: LCMS_PRUSVI_v2022-Training_1999
Exporting: LCMS_PRUSVI_v2022-Training_2000
Exporting: LCMS_PRUSVI_v2022-Training_2001
Exporting: LCMS_PRUSVI_v2022-Training_2002
Exporting: LCMS_PRUSVI_v2022-Training_2003
Exporting: LCMS_PRUSVI_v2022-Training_2004
Exporting: LCMS_PRUSVI_v2022-Training_2005
Exporting: LCMS_PRUSVI_v2022-Training_2006
Exporting: LCMS_PRUSVI_v2022-Training_2007
Exporting: 

In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')

21 tasks ready 2023-08-22 02:57:57
4 tasks running 2023-08-22 02:57:57
Running names:
['LCMS_PRUSVI_v2022-Training_2001', '0:00:44']
['LCMS_PRUSVI_v2022-Training_2000', '0:05:39']
['LCMS_PRUSVI_v2022-Training_1999', '0:09:12']
['LCMS_PRUSVI_v2022-Training_1997', '0:12:41']


21 tasks ready 2023-08-22 02:58:02
4 tasks running 2023-08-22 02:58:02
Running names:
['LCMS_PRUSVI_v2022-Training_2001', '0:00:49']
['LCMS_PRUSVI_v2022-Training_2000', '0:05:44']
['LCMS_PRUSVI_v2022-Training_1999', '0:09:17']
['LCMS_PRUSVI_v2022-Training_1997', '0:12:47']


Serving HTTP on 0.0.0.0 port 4321 (http://0.0.0.0:4321/) ...

Keyboard interrupt received, exiting.


In [10]:
# View final outputs
Map.clearMap()
lcms = ee.ImageCollection(export_assembledLCMSOutputs_collection)

Map.addLayer(lcms,{'opacity':0},'Full Stack Output')
lcms = lcms.map(lambda img:img.set(lcms_viz_dict))
for product in products:Map.addTimeLapse(lcms.select([product]),{'autoViz':True},f'LCMS {product}')

Map.turnOnInspector()
Map.view()

Adding layer: Full Stack Output
Adding layer: LCMS Change
Adding layer: LCMS Land_Cover
Adding layer: LCMS Land_Use
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [22/Aug/2023 03:48:56] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
